In [1]:
from bs4 import BeautifulSoup
import lxml
import html5lib
import re
import json
import uuid
import glob
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.cluster import KMeansClusterer, euclidean_distance
from gensim import corpora, models, utils
from numpy import array
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from pprint import pprint
from gensim import corpora
import re
from os.path import basename
import string
import logging
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.max_columns", 100)

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [4]:
Topics_Name=["Topic 1", "Topic 2", "Topic 3", "Topic 4", "Topic 5", "Topic 6","Topic 7",\
           "Topic 8","Topic 9","Topic 10","Topic 11","Topic 12","Topic 13","Topic 14","Topic 15",\
           "Topic 16","Topic 17","Topic 18","Topic 19","Topic 20","Topic 21","Topic 22","Topic 23",\
           "Topic 24","Topic 25","Topic 26","Topic 27","Topic 28","Topic 29","Topic 30","Topic 31",\
           "Topic 32","Topic 33","Topic 34","Topic 35","Topic 36","Topic 37","Topic 38","Topic 39","Topic 40",\
           "Topic 41","Topic 42","Topic 43","Topic 44","Topic 45","Topic 46","Topic 47","Topic 48","Topic 49","Topic 50"]

In [42]:
Topics_Name_2=["Topic 1", "Topic 2", "Topic 3", "Topic 4", "Topic 5", "Topic 6","Topic 7",\
           "Topic 8","Topic 9","Topic 10","Topic 11","Topic 12","Topic 13","Topic 14","Topic 15",\
           "Topic 16","Topic 17","Topic 18","Topic 19","Topic 20","Topic 21","Topic 22","Topic 23",\
           "Topic 24","Topic 25","Topic 26","Topic 27","Topic 28","Topic 29","Topic 30"]

In [8]:
def find_intersection(TextList,LDAList):
    intersection = list(set(TextList) & set(LDAList))
    return len(intersection)

In [9]:
def clean_LDA_topics(LDATopics):
    addch = "'"
    topics = []
    for i in range(0,len(LDATopics)):
        current_topic  = LDATopics[i][1]
        print(current_topic)
        newwords = []
        split_current_topic = []
        split_current_topic = current_topic.split("+")
        for j in range(0, len(split_current_topic)):
            newword = split_current_topic[j]
            print(split_current_topic)
            if j==0:
                newword = newword[7:-2]
            else:
                newword = newword[8:-2]
                newword = newword[:0] + newword[0:]
            newwords.append(newword)
        topics.append(newwords)
    return topics

In [10]:
def Tokinization(document):
    document = "".join(document)
    tokenizer = RegexpTokenizer(r'\w+')
    intermediate = tokenizer.tokenize(document)
    #stop words
    stop = stopwords.words('english')
    intermediate = [i for i in intermediate if i not in stop]
    #stemming
    lanste = LancasterStemmer()
    intermediate = [lanste.stem(i) for i in intermediate]
    return intermediate

In [11]:
def Preprocess(document):
    #document = document.encode('ascii','ignore')
    #remove http
    intermediate = document
    #join
    intermediate = "".join(document)
    intermediate = re.sub(r'htt\S+', '', intermediate)
    intermediate = re.sub(r'://t.co\S*', '', intermediate)
    # remove rts
    intermediate = re.sub(r'\brt\b','', intermediate)
    # remove digits
    intermediate = re.sub(r'\d+','', intermediate)
    # remove names after @
    # (screen names)
    intermediate = re.sub("@\\w+ *", '', intermediate)
    intermediate = re.sub(r'\b@\b','', intermediate)
    intermediate = unescape(intermediate)
    exclude = set(string.punctuation)
    intermediate = "".join(ch for ch in intermediate if ch is not exclude)
    
    intermediate = Tokinization(intermediate)
    return intermediate

In [12]:
def removeNone(intermediate):
    #remove None Values
    intermediate = ['' if v is None else v for v in intermediate]
    return intermediate

In [13]:
def unescape(s):
    s = s.replace("&gt;", "greater than")
    s = s.replace("&lt;3", "love")
    s = s.replace("&lt;", "less than")
    s = s.replace("&amp;", "and")
    s = s.replace("via", '')
    s = s.replace('\\n', '')
    s = s.replace('\\t', '')
    s = s.replace('\\r', '')
    return s

In [3]:
Daddit = pd.read_pickle('DadditReddit.pkl')

In [16]:
Daddit

,archived,author,author_flair_css_class,author_flair_text,body,controversiality,created_utc,distinguished,downs,edited,gilded,link_id,name,parent_id,removal_reason,score,score_hidden,subreddit,ups
884,True,Hardwarily,None,None,"Excellent post. Plus, I never even *knew* that there was a pampers boycott!",0,2010-08-01 10:01:35,None,0.0,NaT,0,t3_cvzfy,t1_c0vpgyx,t3_cvzfy,NaN,1,False,daddit,1.0
885,True,blot101,None,None,wait a second... yes.... too familiar....\n\n\nyou stole my laptop? c'mon man.,0,2010-08-01 22:23:38,None,0.0,NaT,0,t3_cw6u1,t1_c0vqdne,t3_cw6u1,NaN,7,False,daddit,7.0
886,True,Hardwarily,None,None,Lol! That's all to come for me.,0,2010-08-01 23:06:26,None,0.0,NaT,0,t3_cw6u1,t1_c0vqgpr,t3_cw6u1,NaN,3,False,daddit,3.0
887,True,pigferret,,XX x 2 (6yrs/2yrs),"On the rare occasion I do get it out on the train to or from work, I usually forget about the stickers.\n\nI get a kick out of the mixture of ""wtf"" reactions and ""knowing smile"" reactions.\n\nI wear my My Little Pony stickers with pride.",0,2010-08-01 23:16:07,None,0.0,NaT,0,t3_cw6u1,t1_c0vqhdn,t1_c0vqgpr,NaN,6,False,daddit,6.0
888,True,codepoet,,5yo BALL OF AWESOME,Mine's covered in crayon scribbles. I don't have the heart to clean it. ;),0,2010-08-01 23:40:18,None,0.0,NaT,0,t3_cw6u1,t1_c0vqj32,t3_cw6u1,NaN,3,False,daddit,3.0
889,True,crazyex,,Single father of 3,The inside of the windows in my minivan look like that x 10,0,2010-08-01 23:56:59,None,0.0,NaT,0,t3_cw6u1,t1_c0vqk96,t3_cw6u1,NaN,3,False,daddit,3.0
890,True,blot101,None,None,"that looks like my son. well, the one i gave away in my more foolish (semi) youth \n\nthere is no better time than with a boy that age. it's right when they're smart enough to impress the heck out of you.",0,2010-08-02 00:24:04,None,0.0,NaT,0,t3_cw7pl,t1_c0vqm6o,t3_cw7pl,NaN,4,False,daddit,4.0
891,True,hong_kong_phooey,None,None,"Why yes, [yes it is...](http://i.imgur.com/SLzw0.jpg)",0,2010-08-02 01:06:48,None,0.0,NaT,0,t3_cw6u1,t1_c0vqpdx,t3_cw6u1,NaN,3,False,daddit,3.0
892,True,codepoet,,5yo BALL OF AWESOME,"And that's exactly what he does every day now. :) I don't know what's to come, but I know I love who he is right now. He can't say much, but he can communicate a lot.\n\nI can't imagine being without him. I simply can't. :\",0,2010-08-02 01:15:51,None,0.0,NaT,0,t3_cw7pl,t1_c0vqq2o,t1_c0vqm6o,NaN,3,False,daddit,3.0
893,True,pigferret,,XX x 2 (6yrs/2yrs),Blue's Clues Inside?,0,2010-08-02 01:17:48,None,0.0,NaT,0,t3_cw6u1,t1_c0vqq83,t1_c0vqpdx,NaN,1,False,daddit,1.0


In [17]:
Daddit['body'] = Daddit['body'].str.lower()
bodylist = Daddit['body'].tolist()
NoneRemovedbodylist = removeNone(bodylist)
Daddit['body']=NoneRemovedbodylist
CompleteThread = Daddit.groupby('link_id')['body'].apply(list)

In [26]:
processed_posts = []
#creating threads for each of the users
for post in NoneRemovedbodylist:
    #Preprocessing each of the threads
    processed_posts.append(Preprocess(post))

In [45]:
#running for the rest of the data
processed_threads = []
#creating threads for each of the users
for thread in CompleteThread:
    #Preprocessing each of the threads
    processed_threads.append(Preprocess(thread))

In [24]:
len (CompleteThread)

30290

In [25]:
len(Daddit.author.unique())

38131

In [46]:
texts = processed_threads

['look',
 'lik',
 'first',
 'last',
 'tim',
 'don',
 'gji',
 'nev',
 'happy',
 'hap',
 'yo',
 'stil',
 'know',
 'obvy',
 'although',
 'look',
 'forward',
 'day',
 'act',
 'fav',
 'step',
 'smok',
 'play',
 'video',
 'gam',
 'instead',
 'play',
 'son',
 'par',
 'fail',
 'must',
 'dumbest',
 'thing',
 'read',
 'internet',
 'whol',
 'year',
 'comprehend',
 'fail',
 'gre',
 'thanth',
 'must',
 'dumbest',
 'thing',
 'read',
 'internet',
 'whol',
 'year',
 'nee',
 'read',
 'gre',
 'thancomprehend',
 'fail',
 'nop',
 'play',
 'video',
 'gam',
 'kid',
 'act',
 'spend',
 'tim',
 'spend',
 'much',
 'get',
 'much',
 'tim',
 'nev',
 'know',
 'someth',
 'aw',
 'might',
 'hap',
 'smok',
 'stupid',
 'smok',
 'childr',
 'irrespons',
 'self',
 'stupid',
 'smok',
 'lead',
 'cant',
 'short',
 'lif',
 'leav',
 'childr',
 'without',
 'fath',
 'also',
 'affect',
 'kid',
 'ev',
 'smok',
 'outsid',
 'smok',
 'cling',
 'cloth',
 'dam',
 'childr',
 'lung',
 'say',
 'par',
 'fail',
 'nic',
 'try',
 'afraid',
 'g

In [47]:
# Get token frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token]+=1

In [48]:
tokenized = [[token for token in text if frequency[token]>1]for text in texts]

In [49]:
wordfreq = {}
for text in texts:
    for token in text:
        wordfreq[token]=frequency[token]

In [50]:
higherfreq2 = {k:v for k, v in wordfreq.items() if v>10000}  
higherfreq2

{'act': 17983,
 'also': 24330,
 'alway': 13058,
 'anyth': 10820,
 'around': 16846,
 'ask': 11617,
 'baby': 42735,
 'back': 19635,
 'best': 14801,
 'bet': 16876,
 'big': 10042,
 'bit': 11099,
 'boy': 11705,
 'car': 18580,
 'chang': 14581,
 'child': 19666,
 'com': 23101,
 'could': 17611,
 'dad': 24145,
 'daught': 27777,
 'day': 32943,
 'diff': 11835,
 'easy': 11028,
 'enough': 10318,
 'ev': 35772,
 'every': 15998,
 'famy': 12682,
 'fath': 10148,
 'feel': 24453,
 'fin': 11812,
 'find': 11382,
 'first': 29021,
 'friend': 10854,
 'get': 79343,
 'girl': 11327,
 'giv': 17918,
 'go': 27031,
 'going': 25047,
 'good': 33106,
 'got': 24046,
 'gre': 25556,
 'guy': 10972,
 'hap': 10935,
 'hard': 11838,
 'help': 23110,
 'hom': 15740,
 'hop': 10864,
 'hour': 12963,
 'keep': 15655,
 'kid': 59089,
 'know': 33957,
 'let': 16712,
 'lif': 13907,
 'lik': 66672,
 'littl': 29359,
 'long': 14038,
 'look': 25390,
 'lot': 18184,
 'lov': 24237,
 'mad': 11170,
 'mak': 37278,
 'man': 16459,
 'may': 10505,
 'might'

In [51]:
import operator
x = higherfreq2
sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x

[('stuff', 10026),
 ('big', 10042),
 ('might', 10091),
 ('fath', 10148),
 ('post', 10206),
 ('enough', 10318),
 ('nam', 10392),
 ('may', 10505),
 ('point', 10588),
 ('watch', 10658),
 ('mon', 10812),
 ('anyth', 10820),
 ('friend', 10854),
 ('hop', 10864),
 ('seem', 10888),
 ('hap', 10935),
 ('guy', 10972),
 ('easy', 11028),
 ('bit', 11099),
 ('mad', 11170),
 ('read', 11207),
 ('girl', 11327),
 ('find', 11382),
 ('ask', 11617),
 ('boy', 11705),
 ('fin', 11812),
 ('diff', 11835),
 ('hard', 11838),
 ('said', 12110),
 ('talk', 12379),
 ('new', 12528),
 ('famy', 12682),
 ('tel', 12838),
 ('hour', 12963),
 ('alway', 13058),
 ('though', 13547),
 ('pretty', 13650),
 ('lif', 13907),
 ('long', 14038),
 ('chang', 14581),
 ('best', 14801),
 ('play', 15572),
 ('keep', 15655),
 ('hom', 15740),
 ('every', 15998),
 ('put', 16128),
 ('man', 16459),
 ('let', 16712),
 ('night', 16783),
 ('nev', 16832),
 ('around', 16846),
 ('bet', 16876),
 ('mom', 16883),
 ('two', 17007),
 ('sur', 17282),
 ('could', 1761

In [52]:
dictionary = corpora.Dictionary(texts)
dictionary.save('DadditDict.dict')
print(dictionary)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(32405 unique tokens: ['congrat', 'welcom', 'good', 'beauty', 'baby']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(45694 unique tokens: ['congrat', 'welcom', 'good', 'beauty', 'baby']...)
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary(60947 unique tokens: ['congrat', 'welcom', 'good', 'beauty', 'baby']...)
INFO:gensim.corpora.dictionary:built Dictionary(61349 unique tokens: ['congrat', 'welcom', 'good', 'beauty', 'baby']...) from 30290 documents (total 6616363 corpus positions)
INFO:gensim.utils:saving Dictionary object under DadditDict.dict, separately None
INFO:gensim.utils:saved DadditDict.dict


Dictionary(61349 unique tokens: ['congrat', 'welcom', 'good', 'beauty', 'baby']...)


In [53]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('DadditCorpus.mm', corpus)

INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to DadditCorpus.mm
INFO:gensim.matutils:saving sparse matrix to DadditCorpus.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:PROGRESS: saving document #1000
INFO:gensim.matutils:PROGRESS: saving document #2000
INFO:gensim.matutils:PROGRESS: saving document #3000
INFO:gensim.matutils:PROGRESS: saving document #4000
INFO:gensim.matutils:PROGRESS: saving document #5000
INFO:gensim.matutils:PROGRESS: saving document #6000
INFO:gensim.matutils:PROGRESS: saving document #7000
INFO:gensim.matutils:PROGRESS: saving document #8000
INFO:gensim.matutils:PROGRESS: saving document #9000
INFO:gensim.matutils:PROGRESS: saving document #10000
INFO:gensim.matutils:PROGRESS: saving document #11000
INFO:gensim.matutils:PROGRESS: saving document #12000
INFO:gensim.matutils:PROGRESS: saving document #13000
INFO:gensim.matutils:PROGRESS: saving document #14000
INFO:gensim.matutils:PROGRESS: saving document #15000
I

In [70]:
DadditLDA = models.ldamodel.LdaModel
ldamodelDaddit = DadditLDA(corpus, num_topics=20, id2word=dictionary, passes=20)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.05
INFO:gensim.models.ldamodel:using symmetric eta at 1.630018419208137e-05
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 20 topics, 20 passes over the supplied corpus of 30290 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/30290
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 2000 documents
DEBUG:gensim.models.ldamodel:493/2000 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 30290 documents
INFO:gensim.models.ldamodel:topic #17 (0.050): 0.013*"lik" + 0.012*"kid" + 0.011*"baby" + 0.009*"get" + 0.007*"us" + 0.007*"on" + 0.006*"real" + 0.006*"tim" + 0.006*"mak

In [21]:
ldamodelDaddit.save('DadditLDA.model')

INFO:gensim.utils:saving LdaState object under DadditLDA.model.state, separately None
INFO:gensim.utils:saved DadditLDA.model.state
INFO:gensim.utils:saving LdaModel object under DadditLDA.model, separately ['expElogbeta', 'sstats']
INFO:gensim.utils:storing np array 'expElogbeta' to DadditLDA.model.expElogbeta.npy
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:not storing attribute id2word
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:saved DadditLDA.model


In [71]:
ldamoodelprint = ldamodelDaddit.print_topics(num_topics=20, num_words=30)

INFO:gensim.models.ldamodel:topic #0 (0.050): 0.017*"old" + 0.015*"get" + 0.014*"tim" + 0.013*"lik" + 0.011*"son" + 0.011*"daught" + 0.011*"on" + 0.009*"kid" + 0.008*"year" + 0.008*"littl" + 0.008*"month" + 0.008*"start" + 0.007*"got" + 0.007*"back" + 0.007*"look" + 0.006*"thing" + 0.006*"day" + 0.006*"ev" + 0.006*"say" + 0.006*"around" + 0.006*"hand" + 0.006*"stil" + 0.005*"walk" + 0.005*"first" + 0.005*"us" + 0.005*"mon" + 0.005*"head" + 0.005*"wif" + 0.005*"real" + 0.005*"right"
INFO:gensim.models.ldamodel:topic #1 (0.050): 0.050*"post" + 0.035*"r" + 0.018*"com" + 0.017*"pict" + 0.017*"daddit" + 0.015*"lik" + 0.015*"reddit" + 0.015*"dad" + 0.012*"thank" + 0.012*"link" + 0.012*"see" + 0.011*"sub" + 0.011*"pic" + 0.009*"peopl" + 0.009*"gre" + 0.008*"subreddit" + 0.008*"downvot" + 0.008*"shar" + 0.008*"vot" + 0.007*"us" + 0.007*"facebook" + 0.007*"upvot" + 0.006*"mod" + 0.006*"look" + 0.006*"read" + 0.006*"blog" + 0.006*"new" + 0.006*"pleas" + 0.006*"would" + 0.005*"cont"
INFO:gensim.m

In [72]:
ldamoodelprint

[(0,
  '0.017*"old" + 0.015*"get" + 0.014*"tim" + 0.013*"lik" + 0.011*"son" + 0.011*"daught" + 0.011*"on" + 0.009*"kid" + 0.008*"year" + 0.008*"littl" + 0.008*"month" + 0.008*"start" + 0.007*"got" + 0.007*"back" + 0.007*"look" + 0.006*"thing" + 0.006*"day" + 0.006*"ev" + 0.006*"say" + 0.006*"around" + 0.006*"hand" + 0.006*"stil" + 0.005*"walk" + 0.005*"first" + 0.005*"us" + 0.005*"mon" + 0.005*"head" + 0.005*"wif" + 0.005*"real" + 0.005*"right"'),
 (1,
  '0.050*"post" + 0.035*"r" + 0.018*"com" + 0.017*"pict" + 0.017*"daddit" + 0.015*"lik" + 0.015*"reddit" + 0.015*"dad" + 0.012*"thank" + 0.012*"link" + 0.012*"see" + 0.011*"sub" + 0.011*"pic" + 0.009*"peopl" + 0.009*"gre" + 0.008*"subreddit" + 0.008*"downvot" + 0.008*"shar" + 0.008*"vot" + 0.007*"us" + 0.007*"facebook" + 0.007*"upvot" + 0.006*"mod" + 0.006*"look" + 0.006*"read" + 0.006*"blog" + 0.006*"new" + 0.006*"pleas" + 0.006*"would" + 0.005*"cont"'),
 (2,
  '0.047*"sleep" + 0.024*"night" + 0.020*"baby" + 0.019*"get" + 0.015*"bed" + 

In [57]:
All_topics = clean_LDA_topics(ldamoodelprint)

0.052*"diap" + 0.042*"us" + 0.041*"clo" + 0.021*"dispos" + 0.021*"wash" + 0.011*"sav" + 0.010*"cost" + 0.009*"laundry" + 0.009*"ye" + 0.008*"money" + 0.008*"much" + 0.008*"real" + 0.007*"get" + 0.007*"cream" + 0.007*"brand" + 0.007*"buy" + 0.007*"dry" + 0.006*"ic" + 0.006*"also" + 0.006*"on" + 0.006*"extr" + 0.006*"cheap" + 0.006*"load" + 0.006*"insert" + 0.005*"straw" + 0.005*"siz" + 0.005*"mon" + 0.005*"wat" + 0.005*"would" + 0.004*"every"
['0.052*"diap" ', ' 0.042*"us" ', ' 0.041*"clo" ', ' 0.021*"dispos" ', ' 0.021*"wash" ', ' 0.011*"sav" ', ' 0.010*"cost" ', ' 0.009*"laundry" ', ' 0.009*"ye" ', ' 0.008*"money" ', ' 0.008*"much" ', ' 0.008*"real" ', ' 0.007*"get" ', ' 0.007*"cream" ', ' 0.007*"brand" ', ' 0.007*"buy" ', ' 0.007*"dry" ', ' 0.006*"ic" ', ' 0.006*"also" ', ' 0.006*"on" ', ' 0.006*"extr" ', ' 0.006*"cheap" ', ' 0.006*"load" ', ' 0.006*"insert" ', ' 0.005*"straw" ', ' 0.005*"siz" ', ' 0.005*"mon" ', ' 0.005*"wat" ', ' 0.005*"would" ', ' 0.004*"every"']
['0.052*"diap" ',

In [25]:
ldamodelDaddit.save('DadditLDA.model')

INFO:gensim.utils:saving LdaState object under DadditLDA.model.state, separately None
INFO:gensim.utils:saved DadditLDA.model.state
INFO:gensim.utils:saving LdaModel object under DadditLDA.model, separately ['expElogbeta', 'sstats']
INFO:gensim.utils:storing np array 'expElogbeta' to DadditLDA.model.expElogbeta.npy
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:not storing attribute id2word
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:saved DadditLDA.model


In [82]:
currentList = []
mega_topic_list= [[] for _ in range(len(topiclist)) ]
for i in range(0, len(mega_topic_list)):
    currentList = mega_topic_list[i]
    currentTopic = topiclist[i]
    for j in range (0, len(processed_posts)):
        intersected = find_intersection(processed_posts[j], currentTopic)
        currentList.append(intersected)

In [83]:
for i in range (0,len(Topics_Name)):
    Daddit[Topics_Name[i]]= mega_topic_list[i]

In [94]:
#Hyperparameters for the Daddit Word2Vec
from gensim.models import Word2Vec
dadditmodel = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)

INFO:gensim.utils:loading Word2Vec object from dadditword2vec.model
INFO:gensim.utils:loading wv recursively from dadditword2vec.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:loaded dadditword2vec.model


In [27]:
#Save Daddit Word2Vec model
dadditmodel.save('dadditword2vec.model')

INFO:gensim.utils:saving Word2Vec object under dadditword2vec.model, separately None
INFO:gensim.utils:not storing attribute syn0norm
INFO:gensim.utils:not storing attribute cum_table
INFO:gensim.utils:saved dadditword2vec.model
